In [69]:
!pip install sacremoses

In [70]:
from transformers import BioGptForSequenceClassification, BioGptTokenizer

model_name = "microsoft/biogpt"
model = BioGptForSequenceClassification.from_pretrained(model_name, num_labels=3, problem_type="multi_label_classification")
tokenizer = BioGptTokenizer.from_pretrained(model_name)

Some weights of BioGptForSequenceClassification were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [71]:
import torch

In [72]:
def get_decision(question):
    inputs = tokenizer(question, return_tensors="pt", max_length=512, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
    
    predicted_class_idx = outputs.logits.argmax(dim=-1).item()
    
    return predicted_class_idx

In [73]:
from datasets import load_dataset
dataset = load_dataset("pubmed_qa", "pqa_labeled")

  0%|          | 0/1 [00:00<?, ?it/s]

In [74]:
MAX_TOKENS = 512

def truncate_text_to_fit(text, max_tokens=MAX_TOKENS):
    token_count = len(tokenizer.tokenize(text))
    if token_count > max_tokens - 2:  
        truncated_text = " ".join(text.split()[:-1])
        while len(tokenizer.tokenize(truncated_text)) > max_tokens - 2:
            truncated_text = " ".join(truncated_text.split()[:-1])
    else:
        truncated_text = text
        
    return truncated_text

In [75]:
def create_input_text(sample):
    input_text = "question: "+sample['question'] + " context: " + sample['context']['contexts'][0]
    return {'input_text': input_text}

modified_dataset = dataset['train'].map(create_input_text)

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [76]:
import pandas as pd

# Assuming you have the functions and the modified_dataset loaded

def save_to_csv(indexes, pubids, results, filename='results.csv'):
    """Utility function to save the data to CSV."""
    df = pd.DataFrame({
        'Index': indexes,
        'PubID': pubids,
        'Result': results
    })
    # If file doesn't exist, write header, otherwise append without header
    if not pd.io.common.file_exists(filename):
        df.to_csv(filename, index=False)
    else:
        df.to_csv(filename, mode='a', header=False, index=False)

results = []
pubids = []
indexes = []

for idx, text in enumerate(modified_dataset):
    print(idx + 1)
    
    indexes.append(idx + 1)
    pubids.append(text['pubid'])
    results.append(get_decision(truncate_text_to_fit(text['input_text'])))
    
    if (idx + 1) % 100 == 0:  # Check if it's the 100th, 200th, etc. iteration
        save_to_csv(indexes, pubids, results)
        # Clear the lists to start collecting the next batch of results
        results.clear()
        pubids.clear()
        indexes.clear()

# Save any remaining results (if less than 100) at the end
if results:
    save_to_csv(indexes, pubids, results)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [147]:
data = pd.read_csv('/kaggle/working/results.csv')

In [148]:
data = data.drop('Index', axis=1)

In [149]:
pubids= data["PubID"].values.tolist()

In [150]:
actual={}

for text in dataset['train']:
    if(text['pubid'] in pubids):
        actual[text['pubid']]= text['final_decision']

In [151]:
predictions= dict(data.values)

In [152]:
from sklearn.metrics import accuracy_score, classification_report


if set(predictions.keys()) != set(actual.keys()):
    raise ValueError("Keys in predictions and ground truth do not match!")

In [153]:
y_pred = [predictions[key] for key in sorted(predictions.keys())]
y_true = [actual[key] for key in sorted(actual.keys())]

In [154]:
labels = {
    0 : "yes",
    1 : "no",
    2 : "maybe"
}

In [155]:
y_pred = [labels[key] for key in y_pred]

In [156]:
# Compute accuracy
acc = accuracy_score(y_true, y_pred)
print(f"Accuracy: {acc}")

# Compute detailed classification report
report = classification_report(y_true, y_pred)
print(report)

Accuracy: 0.549
              precision    recall  f1-score   support

       maybe       0.30      0.03      0.05       110
          no       0.25      0.00      0.01       338
         yes       0.55      0.99      0.71       552

    accuracy                           0.55      1000
   macro avg       0.37      0.34      0.25      1000
weighted avg       0.42      0.55      0.40      1000

